<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/earthengine-py-notebooks/tree/master/MachineLearning/cart_classifier.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/earthengine-py-notebooks/blob/master/MachineLearning/cart_classifier.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/earthengine-py-notebooks/blob/master/MachineLearning/cart_classifier.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/giswqs/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/giswqs/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

**Important note**: A key difference between folium and ipyleaflet is that ipyleaflet is built upon ipywidgets and allows bidirectional communication between the front-end and the backend enabling the use of the map to capture user input, while folium is meant for displaying static data only ([source](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a)). Note that [Google Colab](https://colab.research.google.com/) currently does not support ipyleaflet ([source](https://github.com/googlecolab/colabtools/issues/60#issuecomment-596225619)). Therefore, if you are using geemap with Google Colab, you should use [`import geemap.eefolium`](https://github.com/giswqs/geemap/blob/master/geemap/eefolium.py). If you are using geemap with [binder](https://mybinder.org/) or a local Jupyter notebook server, you can use [`import geemap`](https://github.com/giswqs/geemap/blob/master/geemap/geemap.py), which provides more functionalities for capturing user input (e.g., mouse-clicking and moving).

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

## Create an interactive map 
The default basemap is `Google Maps`. [Additional basemaps](https://github.com/giswqs/geemap/blob/master/geemap/basemaps.py) can be added using the `Map.add_basemap()` function. 

In [50]:
import folium
import geemap.eefolium as gmap

Map = gmap.Map(center=[40,-100], zoom=4)
Map.setOptions('HYBRID')
# Map.setOptions('ROADMAP')
# Map.setOptions('SATELLITE')
Map.setOptions('TERRAIN')
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

## Add Earth Engine Python script 

In [ ]:
Map = gmap.Map(center=[40,-100], zoom=4)
# Map.setOptions('HYBRID')
# Map.setOptions('ROADMAP')
# Map.setOptions('SATELLITE')
# Map.setOptions('TERRAIN')



# Add Earth Engine dataset
# Input imagery is a cloud-free Landsat 8 composite.
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')

image = ee.Algorithms.Landsat.simpleComposite(**{
  'collection': l8.filterDate('2018-01-01', '2018-12-31'),
  'asFloat': True
})

# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

# Load training points. The numeric property 'class' stores known labels.
points = ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.cart().train(training, label, bands)

# Classify the image with the same bands used for training.
classified = image.select(bands).classify(trained)

# Display the inputs and the results.
Map.centerObject(points, 11)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 0.4}, 'image')
Map.addLayer(classified,
             {'min': 0, 'max': 2, 'palette': ['red', 'green', 'blue']},
             'classification')



In [91]:
l8.size().getInfo()

1131433

In [67]:
d = training.getInfo()

In [90]:
set([i["properties"]["landcover"] for i in d["features"]])

{0, 1, 2}

In [112]:
points.getInfo()

{'type': 'FeatureCollection',
 'columns': {'landcover': 'Long', 'system:index': 'String'},
 'version': 1569893036369951,
 'id': 'GOOGLE/EE/DEMOS/demo_landcover_labels',
 'properties': {'system:asset_size': 14523},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-122.27096557617189, 37.820452055421086]},
   'id': '00000000000000000003',
   'properties': {'landcover': 0}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-122.2723388671875, 37.828587698949825]},
   'id': '00000000000000000004',
   'properties': {'landcover': 0}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-122.28195190429686, 37.82696064199382]},
   'id': '00000000000000000005',
   'properties': {'landcover': 0}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-122.28126525878908, 37.81882481909333]},
   'id': '00000000000000000006',
   'properties': {'landcover': 0}},
  {'type': 'Feature',
   'geo

In [81]:
d["features"][0]["properties"]

{'B10': 302.1449890136719,
 'B11': 300.9042663574219,
 'B2': 0.13984587788581848,
 'B3': 0.11473792791366577,
 'B4': 0.1099824607372284,
 'B5': 0.11954239010810852,
 'B6': 0.12579503655433655,
 'B7': 0.1057196781039238,
 'landcover': 0}

In [65]:
trained.confusionMatrix().array().getInfo()

[[32, 0, 0], [0, 32, 0], [0, 0, 34]]

In [109]:
classified.getInfo()

{'type': 'Image',
 'bands': [{'id': 'classification',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

## Display Earth Engine data layers 

In [111]:
help(Map.setControlVisibility)

Help on method set_control_visibility in module geemap.eefolium:

set_control_visibility(layerControl=True, fullscreenControl=True, latLngPopup=True) method of geemap.eefolium.Map instance
    Sets the visibility of the controls on the map.
    
    Args:
        layerControl (bool, optional): Whether to show the control that allows the user to toggle layers on/off. Defaults to True.
        fullscreenControl (bool, optional): Whether to show the control that allows the user to make the map full-screen. Defaults to True.
        latLngPopup (bool, optional): Whether to show the control that pops up the Lat/lon when the user clicks on the map. Defaults to True.



In [54]:
# Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map